In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from dataset import TileDataset
from models import TileMLP

2023-10-11 01:15:06.055095: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-11 01:15:06.056974: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-11 01:15:06.083743: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-11 01:15:06.083768: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-11 01:15:06.083784: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [2]:
batch_size = 512
batch_per_file_size = 16
dataset = TileDataset(
    batch_size=batch_size,
    batch_per_file_size=batch_per_file_size,
    build_tfrecords=False)

In [3]:
mlp = TileMLP(
    batch_size, 
    learning_rate=1e-3, 
    batch_per_file_size=batch_per_file_size)

In [4]:
mlp.train(dataset, None)

iteration 500 training loss 2.850186 lr 0.00100
iteration 1000 training loss 2.3755388 lr 0.00100
iteration 1500 training loss 2.2705274 lr 0.00100
iteration 2000 training loss 2.5989246 lr 0.00100
iteration 2500 training loss 1.7902882 lr 0.00100
iteration 3000 training loss 1.8702297 lr 0.00100
iteration 3500 training loss 1.8695171 lr 0.00100
iteration 4000 training loss 2.2868195 lr 0.00100
iteration 4500 training loss 1.7866676 lr 0.00100
iteration 5000 training loss 1.8613592 lr 0.00100
iteration 5500 training loss 1.8637788 lr 0.00090
iteration 6000 training loss 2.502059 lr 0.00090
iteration 6500 training loss 2.045004 lr 0.00090
iteration 7000 training loss 1.8974097 lr 0.00090
iteration 7500 training loss 2.4369378 lr 0.00090
iteration 8000 training loss 1.969621 lr 0.00090
iteration 8500 training loss 2.1023715 lr 0.00090
iteration 9000 training loss 2.0017438 lr 0.00090
iteration 9500 training loss 1.7806907 lr 0.00090
iteration 10000 training loss 2.3087435 lr 0.00090
epoc

## Evaluate validation set

In [ ]:
val_df = mlp.predict_over_dataset(dataset.valid_data, return_labels=True)

In [ ]:
val_df[['prediction', 'target']].describe()

In [ ]:
val_df

In [ ]:
x_lin = np.linspace(0, 3, 100)
plt.plot(x_lin, x_lin, color='orange')

random_sample = val_df.sample(1000)

plt.scatter(
    random_sample.target,
    random_sample.prediction,
    alpha=0.1
)
plt.xlabel('target')
plt.ylabel('prediction')

In [ ]:
random_sample = val_df.sample(5_000)

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.scatter(
    random_sample['target'],
    np.abs(random_sample['target'] - random_sample['prediction']),
    alpha=0.07
)
plt.xlabel('target')
plt.ylabel('abs error')
x_lin = np.linspace(0, 3, 100)
plt.plot(x_lin, x_lin, color='orange')


plt.subplot(1, 2, 2)
plt.scatter(
    random_sample['target'],
    np.square(random_sample['target'] - random_sample['prediction']),
    alpha=0.07
)
plt.xlabel('target')
plt.ylabel('squared error')

In [5]:
def compute_best_5(df):
    top = df.sort_values('prediction').iloc[:5]
    top = top['config_index'].values.tolist()
    top = [str(i) for i in top]
    return ';'.join(top)

In [ ]:
val_prediction = val_df.groupby('ID').apply(compute_best_5)
val_prediction.rename(index=lambda x: x.decode('UTF-8'), inplace=True)
val_prediction

In [ ]:
def tile_score(candidate_configs, tile_dict):
    config_scores = tile_dict['config_runtime'] / tile_dict['config_runtime_normalizers']
    best_runtime = np.min(config_scores)
    best_candidate_runtime = np.min(config_scores[candidate_configs])
    score = 2 - best_candidate_runtime / best_runtime
    return score

In [ ]:
tile_valid_dir = 'predict-ai-model-runtime/npz_all/npz/tile/xla/valid'

scores = []
tile_ids = []
for filename in os.listdir(tile_valid_dir):
    tile_id = 'tile:xla:' + filename.rstrip('.npz')
    tile_dict = dict(np.load(os.path.join(tile_valid_dir, filename)))
    n_configurations = len(tile_dict['config_runtime'])
    candidate_configs = val_prediction[tile_id]
    candidate_configs = [int(i) for i in candidate_configs.split(';')]
    score = tile_score(candidate_configs, tile_dict)
    scores.append(score)
    tile_ids.append(tile_id)

evaluation_result = pd.DataFrame(
    data=np.stack([tile_ids, scores], axis=-1),
    columns=['ID', 'score']
)
evaluation_result

In [ ]:
evaluation_result['score'].astype(float).mean()

## Inference over test set

In [6]:
test_df = mlp.predict_over_dataset(dataset.test_data, return_labels=False)
test_df

,ID,config_index,prediction
0,b'tile:xla:6ec96ca52640832842b7bf32dfb43b5d',2,8.226151
1,b'tile:xla:72e9fc567e22796ad3c058d93c056dbc',326,3.133325
2,b'tile:xla:fd3fa3d0a39c05343eeb8403562f20cc',3551,3.306273
3,b'tile:xla:75121ad778d92722504f620c4c108f45',1023,9.414258
4,b'tile:xla:403b920fef3cc6021cffe9960fdafce1',279,10.113779
...,...,...,...
1420531,b'tile:xla:47037df67e24913fc54f25f01a8e2df6',237,2.976924
1420532,b'tile:xla:47037df67e24913fc54f25f01a8e2df6',5310,1.528257
1420533,b'tile:xla:47037df67e24913fc54f25f01a8e2df6',2796,3.012055
1420534,b'tile:xla:47037df67e24913fc54f25f01a8e2df6',1546,-0.609680


In [7]:
test_prediction = test_df.groupby('ID').apply(compute_best_5)
test_prediction.rename(index=lambda x: x.decode('UTF-8'), inplace=True)
test_prediction

ID
tile:xla:0023795810403f8b0b244d88c901322f     4135;772;3193;2683;2133
tile:xla:005c91ca7a50fffc663678fd44316f04         644;588;502;288;623
tile:xla:0070642211d5a98a16b94f4d7df229fe         1030;61;848;447;958
tile:xla:008191e0c67a6e7a62cde1a3e1d66795        701;610;1077;195;978
tile:xla:008730b43f100be7c2800d7cb89578a4         606;595;757;642;395
                                                       ...           
tile:xla:fe52756188d770ee661f69dcd2688142    2780;8089;6761;7912;7418
tile:xla:fe91ecfc5176e4dea4cec619beb19aeb           168;163;101;15;67
tile:xla:ff9a30ba54b97a48b9f5370a9b1e8cb6             106;33;36;15;21
tile:xla:ffa452493cbec7e4d2fee040879f98ae        778;700;426;815;1616
tile:xla:ffbfd532de0a28c79f4740753fd78006           71;141;118;41;140
Length: 844, dtype: object

In [8]:
pd.DataFrame(test_prediction, columns=['TopConfigs']).to_csv('tile_test_prediction_10_11_01_25.csv')

In [ ]:
plt.hist(np.abs(mlp.dense_layer_1.kernel.numpy().flatten()), bins=50)